In [1]:
#from my_snn.rsnn import RSNN, RSNN_2l, RSNN_2l_ALIF
from my_snn.rsnn_delays import RSNN_d_d, RSNN_d_i, RSNN_d_all
from my_snn.tonic_dataloader import DatasetLoader
from my_snn.utils import train, training_plots, ModelLoader
import torch
device = "cuda:0"

import matplotlib.pyplot as  plt

time_window = 250
batch_size = 256
dataset = 'shd'

DL = DatasetLoader(dataset=dataset, caching='disk', num_workers=0, batch_size=batch_size, time_window=time_window)
test_loader, train_loader = DL.get_dataloaders()

ckpt_dir = 'shd_soa_baseline_iscas'

data = test_loader, train_loader

# save results if best accuracy
import numpy as np
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        snn.test(test_loader)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)

### Training with delays
In this example, we train a 48+48 feedforward network with internal delays (all synapses have multiple delays except those of the input to first-hidden-layer)

In [2]:
hidden = (48, 2, 'f')
depth, stride = 150, 6
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=48, bias=False)
  (f1_f2): Linear(in_features=1200, out_features=48, bias=False)
  (f2_o): Linear(in_features=1200, out_features=20, bias=False)
)
Total params: 115200
Total mult-adds (M): 28.8
training shd250_48x2_d_150_6_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 28.46740
Step [20/31], Loss: 20.18926
Step [30/31], Loss: 12.73931
Time elasped: 64.06670618057251
Epoch [2/50]
Step [10/31], Loss: 8.36053
Step [20/31], Loss: 6.70381
Step [30/31], Loss: 5.54615
Time elasped: 59.645955085754395
Epoch [3/50]
Step [10/31], Loss: 4.74907
Step [20/31], Loss: 3.76352
Step [30/31], Loss: 3.04681
Time elasped: 60.094252586364746
Epoch [4/50]
Step [10/31], Loss: 2.61877
Step [20/31], Loss: 2.30400
Step [30/31], Loss: 2.23184
Time elasped: 59.16747713088989
Epoch [5/50]
Step [10/31], Los

Step [10/31], Loss: 0.16848
Step [20/31], Loss: 0.35943
Step [30/31], Loss: 0.09897
Time elasped: 60.10025882720947
Epoch [50/50]
Step [10/31], Loss: 0.08936
Step [20/31], Loss: 0.08498
Step [30/31], Loss: 0.09814
Time elasped: 60.20612668991089
Test Loss: 1.0323897089276994
Avg spk_count per neuron for all 250 timesteps 20.46929359436035
Test Accuracy of the model on the test samples: 85.449


### Pruning delays
In the previous example 25 delays per synapse were generated.
Here we fine-tune the network and take the 20, 10 and 1 most important delays.

In [ ]:
name = 'shd250_48x2_d_150_6_fs_89697265625'
for k in [20, 10, 1]:
    snn_pooled = ModelLoader(name, 'shd_soa_baseline_iscas', batch_size, 'cuda:0')
    snn_pooled.pool_delays('ho', k=k)
    snn_pooled.test(test_loader, dropout = 0.2)
    snn_pooled.train_loss = []
    snn_pooled.test_loss = []
    snn_pooled.acc = []
    snn_pooled.modelname = f'{name}_pooled_{k}'
    train(snn_pooled, data, 1e-3, 20, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
    torch.cuda.empty_cache()

delays: [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 0.33821641760213034
Avg spk_count per neuron for all 250 timesteps 20.236309051513672
Test Accuracy of the model on the test samples: 91.553
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=48, bias=False)
  (f1_f2): Linear(in_features=1200, out_features=48, bias=False)
  (f2_o): Linear(in_features=1200, out_features=20, bias=False)
)
Total params: 115200
Total mult-adds (M): 28.8
training shd250_48x2_d_150_6_fs_89697265625_pooled_20 for 20 epochs...
Epoch [1/20]
Step [10/31], Loss: 0.22376
Step [20/31], Loss: 0.21206
Step [30/31], Loss: 0.04421
Time elasped: 59.454264640808105
Epoch [2/20]
Step [10/31], Loss: 0.02840
Step [20/31], Loss: 0.02699
Step [30/31], Loss: 0.00927
Time elasped: 59.5541934967041
Epoch [3/20]
Step [10/31], Loss: 0.01312
Step [20/

RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=48, bias=False)
  (f1_f2): Linear(in_features=1200, out_features=48, bias=False)
  (f2_o): Linear(in_features=1200, out_features=20, bias=False)
)
Total params: 115200
Total mult-adds (M): 28.8
training shd250_48x2_d_150_6_fs_89697265625_pooled_1 for 20 epochs...
Epoch [1/20]
Step [10/31], Loss: 27.72841
Step [20/31], Loss: 26.40421
Step [30/31], Loss: 25.33061
Time elasped: 61.25180625915527
Epoch [2/20]
Step [10/31], Loss: 24.34526
Step [20/31], Loss: 23.78618
Step [30/31], Loss: 23.51901
Time elasped: 59.36224603652954
Epoch [3/20]
Step [10/31], Loss: 22.95551
Step [20/31], Loss: 22.72355
Step [30/31], Loss: 22.20191
Time elasped: 60.76542592048645
Epoch [4/20]
Step [10/31], Loss: 21.99911
